In [1]:
## importing relevant libraries
import pandas as pd
import numpy as np
import datetime

In [2]:
## reading the excel and csv files accordingly, this could be automated using loop if we have more numners of file in folder
data1=pd.read_excel("AUS.xlsx")
data2=pd.read_csv("IND.csv")
data3=pd.read_csv("USA.csv")

In [3]:
## adding a country column in each file to differentiate them after combining
data1['country']='AUS'
data2['country']='IND'
data3['country']='USA'

In [4]:
## as seen in this there was a pattern in date , written in reverse, so extracting year from right with four characters,
## similarly done for date and month as well
data3['year']=data3.VaccinationDate.astype(str).str[-4:]
data3['date']=data3.VaccinationDate.astype(str).str[-6:-4]
data3['month']=data3.VaccinationDate.astype(str).str[:-6]
data3.month=data3.month.str.zfill(2)
data3['VaccinationDate_modified']=data3.year+"-"+data3.month+"-"+data3.date
data3['VaccinationDate']=data3.VaccinationDate_modified
data3.drop(columns=['year','date','month','VaccinationDate_modified'],inplace=True)
data3.VaccinationDate=pd.to_datetime(data3.VaccinationDate)
data3

,ID,Name,VaccinationType,VaccinationDate,country
0,1,Sam,EFG,2022-06-15,USA
1,2,John,XYZ,2022-01-05,USA
2,3,Mike,ABC,2021-12-28,USA


In [5]:
## converting to date format and excluding incoorect date, at large scale this approcah will not be used , i will have to 
## split month and year and date and check if month lies only between values 1 and 12,
## for dates it would be 1 to 31
## for year it will in for digit but not greater then 2022
## will use to map date and make it or two digit as 2 will not work so will make it 02 , like did in data3
data1['A']=pd.to_datetime(data1['Date of Vaccination'],errors='coerce')
data1['invalid_date']=np.where(data1.A.isna(),data1['Date of Vaccination'],'')
data1['Date of Vaccination']=data1['A']
data1.drop(columns=['A'],inplace=True)
data1

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination,country,invalid_date
0,1,Mike,LMN,NaT,2022-05-11,AUS,
1,2,Jonnathan,XYZ,1997-12-13,NaT,AUS,2021-13-13
2,3,Cristina,ABC,1998-03-12,2022-03-12,AUS,


In [6]:
## combining data to singlke file by making a few changes in column name
data2.VaccinationDate=pd.to_datetime(data2.VaccinationDate)
data2.DOB=pd.to_datetime(data2.DOB)
data=data1.merge(data2.rename(columns={'ID':'Unique ID','Name':'Patient Name','DOB':'Date of Birth'
                                       ,'VaccinationType':'Vaccine Type','VaccinationDate':'Date of Vaccination'}),how='outer')
data=data.merge(data3.rename(columns={'ID':'Unique ID','Name':'Patient Name',
                                     'VaccinationType':'Vaccine Type','VaccinationDate':'Date of Vaccination'}),how='outer')
data

,Unique ID,Patient Name,Vaccine Type,Date of Birth,Date of Vaccination,country,invalid_date,Free or Paid
0,1,Mike,LMN,NaT,2022-05-11,AUS,,NaN
1,2,Jonnathan,XYZ,1997-12-13,NaT,AUS,2021-13-13,NaN
2,3,Cristina,ABC,1998-03-12,2022-03-12,AUS,,NaN
3,1,Vikas,XYZ,1998-12-01,2022-01-01,IND,NaN,F
4,2,Rahul,ABC,1982-08-13,2022-03-05,IND,NaN,P
5,3,Sameer,ABC,1952-08-13,2022-02-20,IND,NaN,F
6,1,Sam,EFG,NaT,2022-06-15,USA,NaN,NaN
7,2,John,XYZ,NaT,2022-01-05,USA,NaN,NaN
8,3,Mike,ABC,NaT,2021-12-28,USA,NaN,NaN


### Mectric 1
CountryName, VaccinationType, No. of vaccinations

In [7]:
vaccine_type_country=pd.DataFrame(data.groupby(['country','Vaccine Type']).count().reset_index())
vaccine_type_country=vaccine_type_country[['country','Vaccine Type','Unique ID']]
vaccine_type_country

,country,Vaccine Type,Unique ID
0,AUS,ABC,1
1,AUS,LMN,1
2,AUS,XYZ,1
3,IND,ABC,2
4,IND,XYZ,1
5,USA,ABC,1
6,USA,EFG,1
7,USA,XYZ,1


In [8]:
for index,row in vaccine_type_country.iterrows():
  print("In Country",row[0],"with Vaccine Type",row[1],"the total number of vaccinations given are",row[2])

In Country AUS with Vaccine Type ABC the total number of vaccinations given are 1
In Country AUS with Vaccine Type LMN the total number of vaccinations given are 1
In Country AUS with Vaccine Type XYZ the total number of vaccinations given are 1
In Country IND with Vaccine Type ABC the total number of vaccinations given are 2
In Country IND with Vaccine Type XYZ the total number of vaccinations given are 1
In Country USA with Vaccine Type ABC the total number of vaccinations given are 1
In Country USA with Vaccine Type EFG the total number of vaccinations given are 1
In Country USA with Vaccine Type XYZ the total number of vaccinations given are 1


### Assuming
AUS population being 10, IND population being 30 and USA population being 20 and worldwide population being 100

### Metric 2 
CountryName, % Vaccinated

In [9]:
percentage_vaccine=data.copy()
percentage_vaccine=pd.DataFrame(percentage_vaccine.groupby(['country']).count().reset_index())
percentage_vaccine=percentage_vaccine[['country','Unique ID']]
percentage_vaccine['percentage_of_people_vaccinated']=np.where(percentage_vaccine.country=='AUS',percentage_vaccine['Unique ID']/10,
                                                              np.where(percentage_vaccine.country=='IND',percentage_vaccine['Unique ID']/30,
                                                                      percentage_vaccine['Unique ID']/20))
percentage_vaccine['percentage_of_people_vaccinated']=round((percentage_vaccine['percentage_of_people_vaccinated']*100),2).astype(str)+"%"
percentage_vaccine.drop(columns=['Unique ID'],inplace=True)
percentage_vaccine

,country,percentage_of_people_vaccinated
0,AUS,30.0%
1,IND,10.0%
2,USA,15.0%


In [10]:
for index,row in percentage_vaccine.iterrows():
  print("In Country",row[0],"percentage of vaccinated people are",row[1])

In Country AUS percentage of vaccinated people are 30.0%
In Country IND percentage of vaccinated people are 10.0%
In Country USA percentage of vaccinated people are 15.0%


### Metric 3
CountryName, % Contribution

In [11]:
world_level_vaccine=data.copy()
world_level_vaccine=pd.DataFrame(world_level_vaccine.groupby(['country']).count().reset_index())
world_level_vaccine=world_level_vaccine[['country','Unique ID']]
world_level_vaccine['world_level_percentage']=(round(world_level_vaccine['Unique ID']/100,2)*100).astype(str)+"%"
world_level_vaccine=world_level_vaccine[['country','world_level_percentage']]
world_level_vaccine

,country,world_level_percentage
0,AUS,3.0%
1,IND,3.0%
2,USA,3.0%


In [12]:
for index,row in world_level_vaccine.iterrows():
  print("In Country",row[0],"percentage of vaccinated people with respect to population of world",row[1])

In Country AUS percentage of vaccinated people with respect to population of world 3.0%
In Country IND percentage of vaccinated people with respect to population of world 3.0%
In Country USA percentage of vaccinated people with respect to population of world 3.0%


In [13]:
total_among_all_three_countries_people_vaccinated=((data['Unique ID'].count()/100)*100).astype(str)+"%"
print('total number of people vaccinated with respect to world among all these three countries are : ',total_among_all_three_countries_people_vaccinated)

total number of people vaccinated with respect to world among all these three countries are :  9.0%
